In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, TextVectorization, Embedding, Dense, Dropout, GlobalAveragePooling1D

from tensorflow.keras import layers, losses

print(tf.__version__)

2.19.0-dev20250105


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-01-06 08:38:06--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.005s  

2025-01-06 08:38:06 (71.0 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2025-01-06 08:38:06--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [ ]:
train_data = pd.read_csv(train_file_path, sep='\t', header=None, names=['type', 'message'])
test_data = pd.read_csv(test_file_path, sep='\t', header=None, names=['type', 'message'])

## PreProcess Data

In [ ]:
train_data.head()

,type,message
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...


In [ ]:
train_data['type'].unique()

array(['ham', 'spam'], dtype=object)

In [ ]:
len(train_data[train_data['type'] == 'ham']), len(train_data[train_data['type'] == 'spam'])

(3619, 560)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=42)
train_data, train_labels = rus.fit_resample(train_data[['message']], train_data['type'])

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(


In [ ]:
train_labels.value_counts()

,count
type,
ham,560
spam,560


In [ ]:
train_data.shape

(1120, 1)

In [ ]:
max_features = 10000
sequence_length = 250

vector_layer = TextVectorization(max_tokens= max_features,
                                 standardize="lower_and_strip_punctuation",
                                 output_mode='int',
                                 output_sequence_length=sequence_length)
vector_layer.adapt(train_data)

AttributeError: module 'ml_dtypes' has no attribute 'float8_e3m4'


In [ ]:
len(vector_layer.get_vocabulary())

4178

In [ ]:
test_label = test_data['type']
test_data =  test_data['message']

In [ ]:
train_sequence = vector_layer(train_data)
test_sequence = vector_layer(test_data)

In [ ]:
train_labels = np.where(train_labels=='spam', 1, 0)
test_label = np.where(test_label=='spam', 1, 0)

## Model Building & Training

In [ ]:
model = keras.Sequential([
    layers.Embedding(input_dim= max_features + 1,
                     output_dim= 128),
    #layers.Dropout(0.2),
    layers.GlobalAveragePooling1D(),
    layers.Dense(64,activation='relu'),
    #layers.Dropout(0.2),
    layers.Dense(16,activation='relu'),
    layers.Dense(1,activation='sigmoid')
])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_1           │ ?                           │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(loss=losses.BinaryCrossentropy(from_logits = True),
              optimizer='adam',
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.5),
                       tf.metrics.Precision(),
                       tf.metrics.Recall()])
checkpoint = keras.callbacks.ModelCheckpoint("best_model.keras",
                                             monitor = 'val_loss',
                                             save_best_only = True,
                                             mode = 'min')

In [ ]:
model.fit(x = train_sequence,
          y = train_labels,
          validation_data = (test_sequence,test_label),
          epochs = 15,# total epoches trained 5+10 = 15
          callbacks= [checkpoint])

Epoch 1/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - binary_accuracy: 0.9599 - loss: 0.1072 - precision_1: 0.9690 - recall_1: 0.9538 - val_binary_accuracy: 0.9361 - val_loss: 0.2009 - val_precision_1: 0.6885 - val_recall_1: 0.9572
Epoch 2/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - binary_accuracy: 0.9729 - loss: 0.0731 - precision_1: 0.9804 - recall_1: 0.9638 - val_binary_accuracy: 0.9741 - val_loss: 0.0963 - val_precision_1: 0.8912 - val_recall_1: 0.9198
Epoch 3/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - binary_accuracy: 0.9554 - loss: 0.1106 - precision_1: 0.9784 - recall_1: 0.9346 - val_binary_accuracy: 0.9670 - val_loss: 0.1097 - val_precision_1: 0.8439 - val_recall_1: 0.9251
Epoch 4/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - binary_accuracy: 0.9783 - loss: 0.0712 - precision_1: 0.9889 - recall_1: 0.9672 - val_binary_accuracy: 0.9497 - val_loss: 0.1545 - val_precision_1: 0.7468 - val_recall_1: 0.9465
Epoch 5/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - binary_accuracy: 0.9833

## Model Evaluation

In [ ]:
from tensorflow.keras.models import load_model

new_model = load_model('best_model.keras')

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  text_sequence = vector_layer([pred_text])
  prediction = new_model.predict(text_sequence)
  if prediction[0][0] > 0.5:
    return [prediction[0][0],'spam']
  else:
    return [prediction[0][0],'ham']

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
[0.07288352, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
You passed the challenge. Great job!
